# Setup

In [2]:
%pip install transformers==4.48.3
%pip install datasets==3.2.0
%pip install optimum==1.24.0
%pip install openai==1.61.0
%pip install wandb
%pip install pydantic_settings
%pip install json_repair==0.29.1
%pip install accelerate==0.26.0


  Using cached transformers-4.48.3-py3-none-any.whl.metadata (44 kB)
  Using cached regex-2026.1.15-cp310-cp310-win_amd64.whl.metadata (41 kB)
  Using cached tokenizers-0.21.4-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.7.0-cp38-abi3-win_amd64.whl.metadata (4.2 kB)
  Using cached tqdm-4.67.3-py3-none-any.whl.metadata (57 kB)
Using cached transformers-4.48.3-py3-none-any.whl (9.7 MB)
   ---------------------------------------- 0.0/566.4 kB ? eta -:--:--
   ---------------------------------------- 0.0/566.4 kB ? eta -:--:--
   ------------------ --------------------- 262.1/566.4 kB ? eta -:--:--
   ---------------------------------------- 566.4/566.4 kB 1.4 MB/s  0:00:00
Using cached tokenizers-0.21.4-cp39-abi3-win_amd64.whl (2.5 MB)
Using cached regex-2026.1.15-cp310-cp310-win_amd64.whl (277 kB)
Using cached safetensors-0.7.0-cp38-abi3-win_amd64.whl (341 kB)
Using cached tqdm-4.67.3-py3-none-any.whl (78 kB)

   ---------------------------------------- 0/6 [tq

In [ ]:
!git clone --depth 1 https://github.com/hiyouga/LlamaFactory.git
!cd LlamaFactory && pip install -e .

# Imports

In [ ]:
import os
import json
import wandb
import torch
import random
import requests
import json_repair 
from os.path import join
from tqdm.auto import tqdm
from datetime import datetime
from pydantic import BaseModel,Field
from Helper.config import get_settings
from typing import List, Optional , Literal
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [ ]:
import torch
print("Torch:", torch.__version__)
print("CUDA:", torch.version.cuda)
print("Available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))


Torch: 2.5.1+cu121
CUDA: 12.1
Available: True
GPU: NVIDIA GeForce RTX 3050 Laptop GPU


# Sittings

In [30]:
def parse_json(text):
    try:
        return json_repair.loads(text)
    except:
        return None 

In [3]:
settings = get_settings()
wandb.login(key=settings.WANDB_API_KEY)
!huggingface-cli login --token {settings.HUGGINGFACE_API_KEY}

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: WARNING [wandb.login()] Changing session credentials to explicit value for https://api.wandb.ai.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\a1hmm\_netrc


⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `FineTuning` has been saved to C:\Users\a1hmm\.cache\huggingface\stored_tokens
Your token has been saved to C:\Users\a1hmm\.cache\huggingface\token
Login successful.
The current active token is: `FineTuning`


In [4]:
data_dir = "src/Data"
base_model_id = "Qwen/Qwen2.5-1.5B-Instruct"
device = "cuda"
torch_dtype = None

# Sample

In [5]:
story = """
ذكرت مجلة فوربس أن العائلة تلعب دورا محوريا في تشكيل علاقة الأفراد بالمال،
 حيث تتأثر هذه العلاقة بأنماط السلوك المالي المتوارثة عبر الأجيال.

التقرير الذي يستند إلى أبحاث الأستاذ الجامعي شاين إنيت حول
الرفاه المالي يوضح أن لكل شخص "شخصية مالية" تتحدد وفقا لطريقة
 تفاعله مع المال، والتي تتأثر بشكل مباشر بتربية الأسرة وتجارب الطفولة.

 الأبعاد الثلاثة للعلاقة بالمال
بحسب الدراسة، هناك ثلاثة أبعاد رئيسية تشكّل علاقتنا بالمال:

الاكتساب (A): يميل الأفراد الذين ينتمون لهذا
 البعد إلى اعتبار المال سلعة قابلة للجمع، حيث يرون
في تحقيق الثروة هدفا بحد ذاته. والجانب السلبي لهذا
 النمط هو إمكانية التحول إلى هوس بالثروة أو العكس،
 أي رفض تام لاكتساب المال باعتباره مصدرا للفساد.

الاستخدام (U): يرى هؤلاء الأشخاص المال أداة للتمتع بالحياة، حيث يربطون قيمته بقدرته على توفير
المتعة والراحة. ومع ذلك، قد يصبح
البعض مدمنا على الإنفاق، في حين يتجه آخرون إلى التقشف المفرط خوفا من المستقبل.

الإدارة (M): أصحاب هذا النمط يعتبرون المال مسؤولية تتطلب التخطيط الدقيق. لكن في بعض الحالات،
 قد يتحول الأمر إلى هوس مفرط بإدارة الإنفاق، مما يؤثر سلبا على العلاقات الشخصية.

 كيف تؤثر العائلة على علاقتنا بالمال؟
يشير التقرير إلى أن التجارب الأسرية تلعب دورا رئيسيا في تحديد
 "الشخصية المالية" لكل فرد، على سبيل المثال، إذا كان أحد الوالدين يعتمد على المال
كمكافأة للسلوك الجيد، فقد يتبنى الطفل لاحقا النمط نفسه في حياته البالغة.

لتحليل هذه التأثيرات بشكل دقيق، طورت رابطة العلاج المالي
(Financial Therapy Association) أداة تسمى مخطط الجينوم المالي (Money Genogram)،
وهو نموذج يُستخدم لتحديد الأنماط المالية داخل العائلة.

تتضمن هذه الأداة:

رسم شجرة عائلية.
تصنيف أفراد العائلة وفقا للأبعاد الثلاثة للعلاقة بالمال (A ،U ،M).
تحديد ما إذا كان السلوك المالي لكل فرد صحيا (+) أو غير صحي (-).
على سبيل المثال، إذا نشأ شخص في عائلة
اعتادت على الإنفاق المفرط، فقد يكون لديه ميل قوي إلى اتباع النمط نفسه،
 أو العكس تماما، حيث يصبح مقتصدا بشكل مبالغ فيه كرد فعل نفسي.
"""

In [6]:
StoryCategory = Literal[
    "politics", "sports", "art", "technology", "economy",
    "health", "entertainment", "science",
    "not_specified"
]

EntityType = Literal[
    "person-male", "person-female", "location", "organization", "event", "time",
    "quantity", "money", "product", "law", "disease", "artifact", "not_specified"
]

class Entity(BaseModel):
    entity_value: str = Field(..., description="The actual name or value of the entity.")
    entity_type: EntityType = Field(..., description="The type of recognized entity.")

class NewsDetails(BaseModel):
    story_title: str = Field(..., min_length=5, max_length=300,
                             description="A fully informative and SEO optimized title of the story.")

    story_keywords: List[str] = Field(..., min_length=1,
                                      description="Relevant keywords associated with the story.")

    story_summary: List[str] = Field(
                                    ..., min_length=1, max_length=5,
                                    description="Summarized key points about the story (1-5 points)."
                                )

    story_category: StoryCategory = Field(..., description="Category of the news story.")

    story_entities: List[Entity] = Field(..., min_length=1, max_length=10,
                                        description="List of identified entities in the story.")

In [25]:
details_extraction_messages = [
    {
        "role": "system",
        "content":"\n".join([
            "You are an NLP data paraser.",
            "You will be provided by an Arabic text associated with a Pydantic scheme.",
            "Generate the ouptut in the same story language.",
            "You have to extract JSON details from text according the Pydantic details.",
            "Extract details as mentioned in text.",
            "Do not generate any introduction or conclusion."

        ])
    },
    {
        "role": "user",
        "content": "\n".join([
            "## Story:",
            story.strip(),
            "",
            "## Pydantic Schema:",
            json.dumps(NewsDetails.model_json_schema(), ensure_ascii=False),
            "",
            "## Story Details:",
            "```json",
        ])
    }

]

# Translation

In [8]:
class TranslatedStory(BaseModel):
    translated_title: str = Field(..., description="Suggested translated news story title.")
    translated_content: str = Field(..., description="Translated content of the news story ")

targeted_lang = "English"

translation_messages = [
    {
        "role": "system",
        "content":"\n".join([
            "You are a professional translator.",
            "You will be provided by an Arabic text.",
            "You have to translate the text into the `Targeted Language`.",
            "Follow the provided Scheme to generate a JSON",
            "Do not generate any introduction or conclusion."

        ])
    },
    {
        "role": "user",
        "content":  "\n".join([
            "## Story:",
            story.strip(),
            "",

            "## Pydantic Details:",
            json.dumps( TranslatedStory.model_json_schema(), ensure_ascii=False ),
            "",

            "## Targeted Language:",
            targeted_lang,
            "",

            "## Translated Story:",
            "```json"
        ])
    }
]

# Evaluation

In [9]:
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    torch_dtype=torch_dtype,
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id)

In [8]:
text = tokenizer.apply_chat_template(
    details_extraction_messages,
    tokenize =False,
    add_generation_prompt=True
)

model_inputs = tokenizer(
    [text],
    return_tensors="pt",    
).to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=1024,
    do_sample=False,
    top_k= None,
    top_p= None,
    temperature=None,
)
generated_ids = [
    output_ids[len(input_ids):]
    for input_ids, output_ids in zip(
        model_inputs.input_ids,
        generated_ids
    )
]

response = tokenizer.batch_decode(
    generated_ids,
    skip_special_tokens=True
)[0]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [ ]:
print(response)

In [9]:
text = tokenizer.apply_chat_template(
    translation_messages,
    tokenize =False,
    add_generation_prompt=True
)

model_inputs = tokenizer(
    [text],
    return_tensors="pt",    
).to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=1024,
    do_sample=False,
    top_k= None,
    top_p= None,
    temperature=None,
)
generated_ids = [
    output_ids[len(input_ids):]
    for input_ids, output_ids in zip(
        model_inputs.input_ids,
        generated_ids
    )
]

response = tokenizer.batch_decode(
    generated_ids,
    skip_special_tokens=True
)[0]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [10]:
print(response)

{
  "translated_title": "Forbes Magazine: Family Plays a Central Role in Forming Individuals' Financial Relationships",
  "translated_content": "According to Forbes magazine, family plays a crucial role in shaping individuals' financial relationships, as these relationships are influenced by inherited behavioral patterns across generations."
}


In [10]:
from  openai import OpenAI

client = OpenAI(api_key=settings.OPENAI_API_KEY)

openai_model_id = settings.OPENAI_MODEL

In [14]:
chat_messages = client.chat.completions.create(
    messages = details_extraction_messages,
    model=openai_model_id,
    temperature=0.2,
)
print(chat_messages.choices[0].message.content)

{
  "story_title": "تأثير العائلة على العلاقة بالمال",
  "story_keywords": [
    "العائلة",
    "المال",
    "الشخصية المالية",
    "السلوك المالي",
    "الرفاه المالي"
  ],
  "story_summary": [
    "تلعب العائلة دورا محوريا في تشكيل علاقة الأفراد بالمال.",
    "هناك ثلاثة أبعاد رئيسية للعلاقة بالمال: الاكتساب، الاستخدام، والإدارة.",
    "التجارب الأسرية تحدد 'الشخصية المالية' لكل فرد.",
    "تم تطوير أداة مخطط الجينوم المالي لتحليل الأنماط المالية داخل العائلة."
  ],
  "story_category": "economy",
  "story_entities": [
    {
      "entity_value": "مجلة فوربس",
      "entity_type": "organization"
    },
    {
      "entity_value": "شاين إنيت",
      "entity_type": "person-male"
    },
    {
      "entity_value": "رابطة العلاج المالي",
      "entity_type": "organization"
    },
    {
      "entity_value": "مخطط الجينوم المالي",
      "entity_type": "product"
    }
  ]
}


In [15]:
chat_messages = client.chat.completions.create(
    messages = translation_messages,
    model=openai_model_id,
    temperature=0.2,
)
print(chat_messages.choices[0].message.content)

{
  "translated_title": "The Impact of Family on Financial Relationships",
  "translated_content": "Forbes magazine stated that family plays a pivotal role in shaping individuals' relationships with money, as this relationship is influenced by inherited financial behavior patterns across generations.\n\nThe report, based on research by university professor Shane Enright on financial well-being, explains that each person has a \"financial personality\" determined by how they interact with money, which is directly affected by family upbringing and childhood experiences.\n\nThe three dimensions of the relationship with money\nAccording to the study, there are three main dimensions that shape our relationship with money:\n\nAcquisition (A): Individuals belonging to this dimension tend to view money as a commodity to be accumulated, seeing wealth accumulation as a goal in itself. The downside of this pattern is the potential to develop an obsession with wealth or, conversely, a complete rej

# Knwoledge Distillation

In [22]:
data_path = "Data/news-sample.jsonl"
row_data = []
for line in open(data_path):
    if line.strip()=="":
        continue
    row_data.append(
        json.loads(line.strip())
    )

random.Random(101).shuffle(row_data)

print(f"Total Rows: {len(row_data)}")

Total Rows: 2400


In [ ]:
# Data Extraction and Saving to JSONL
price_per_1m_input_tokens = 0.150
price_per_1m_output_tokens = 0.600

prompt_tokens = 0
completion_tokens = 0

saveto_path = "Data/sft.jsonl"
itr = 0
for story in tqdm(row_data):
    sample_details_extraction_messages = [
    {
        "role": "system",
        "content":"\n".join([
            "You are an NLP data paraser.",
            "You will be provided by an Arabic text associated with a Pydantic scheme.",
            "Generate the ouptut in the same story language.",
            "You have to extract JSON details from text according the Pydantic details.",
            "Extract details as mentioned in text.",
            "Do not generate any introduction or conclusion."

        ])
    },
    {
        "role": "user",
        "content": "\n".join([
            "## Story:",
            story['content'].strip(),
            "",
            "## Pydantic Schema:",
            json.dumps(NewsDetails.model_json_schema(), ensure_ascii=False),
            "",
            "## Story Details:",
            "```json",
        ])
    }

    ]
    response = client.chat.completions.create(
                    messages = sample_details_extraction_messages,
                    model=openai_model_id,
                    temperature=0.2,
    )
    if response.choices[0].finish_reason != "stop":
        prompt_tokens+= response.usage.prompt_tokens
        print("Warning: Incomplete response detected.")
        continue

    llm_resp_dict = response.choices[0].message.content
    parsed_output = parse_json(llm_resp_dict)
    if parsed_output is None:
        continue

    with open(saveto_path, "a", encoding="utf-8") as fout:
        fout.write(
            json.dumps(
                {
                    "story": story['content'].strip(),
                    "task": "Extract the story details into a JSON",
                    "output_schema": json.dumps(NewsDetails.model_json_schema(), ensure_ascii=False),
                    "response": llm_resp_dict,
                },
                ensure_ascii=False , default=str
            )+"\n"
        )
    itr+=1
    prompt_tokens+= response.usage.prompt_tokens
    completion_tokens+= response.usage.completion_tokens
    if itr % 50 ==0:
        cost_input = (prompt_tokens / 1000000) * price_per_1m_input_tokens
        cost_output = (completion_tokens / 1000000) * price_per_1m_output_tokens
        total_cost = cost_input + cost_output
        print(f"Processed Samples: {itr}, Total Cost: ${total_cost:.4f}")

In [ ]:
# Data Translation and Saving to JSONL
price_per_1m_input_tokens = 0.150
price_per_1m_output_tokens = 0.600

prompt_tokens = 0
completion_tokens = 0

save_to = join(data_dir, "datasets", "sft.jsonl")

ix = 0
for story in tqdm(row_data):

    for targeted_lang in ["English", "French"]:
        sample_translation_messages = [
            {
                "role": "system",
                "content": "\n".join([
                    "You are a professional translator.",
                    "You will be provided by an Arabic text.",
                    "You have to translate the text into the `Targeted Language`.",
                    "Follow the provided Scheme to generate a JSON",
                    "Do not generate any introduction or conclusion."
                ])
            },
            {
                "role": "user",
                "content": "\n".join([
                    "## Pydantic Details:",
                    json.dumps( TranslatedStory.model_json_schema(), ensure_ascii=False ),
                    "",

                    "## Targeted Language or Dialect:",
                    targeted_lang,
                    "",

                    "## Story:",
                    story['content'].strip(),
                    "",

                    "## Translated Story:",
                    "```json"
                ])
            }
        ]

        response = client.chat.completions.create(
                                messages=sample_translation_messages,
                                model=openai_model_id,
                                temperature=0.2,
                            )

        if response.choices[0].finish_reason != "stop":
            prompt_tokens += response.usage.prompt_tokens
            continue

        llm_response = response.choices[0].message.content
        llm_resp_dict = parse_json(llm_response)

        if not llm_resp_dict:
            continue

        with open(save_to, "a", encoding="utf8") as dest:
            dest.write(json.dumps({
                "id": ix,
                "story": story['content'].strip(),

                "output_scheme": json.dumps( TranslatedStory.model_json_schema(), ensure_ascii=False ),
                "task": f"You have to translate the story content into {targeted_lang} associated with a title into a JSON.",

                "response": llm_resp_dict,
            }, ensure_ascii=False, default=str)  + "\n" )

        ix += 1
        prompt_tokens += response.usage.prompt_tokens
        completion_tokens += response.usage.completion_tokens

        if(ix % 3) == 0:
            cost_input = (prompt_tokens / 1_000_000) * price_per_1m_input_tokens
            cost_output = (completion_tokens / 1_000_000) * price_per_1m_output_tokens
            total_cost = cost_input + cost_output

            print(f"Iteration {ix}: Total Cost = ${total_cost:.4f} ")

# Format FineTuning Datasets

In [31]:
sft_data_path = "Data/sft.jsonl"
llm_fine_tuning_data = []
for line in open(sft_data_path, encoding="utf-8"):
    if line.strip()=="":
        continue
    rec = json.loads(line.strip())
    break

In [32]:
rec

{'id': 0,
 'story': 'ظلت أسعار المنتجين بالولايات المتحدة دون تغيير في سبتمبرأيلول الماضي مدفوعة بانخفاض تكاليف البنزين، مما يشير إلى تقدم نحو تضخم أقل حدة، وهو ما يدعم توقعات خفض مجلس الاحتياطي الاتحادي المركزي الأميركي أسعار الفائدة مجددا الشهر المقبل. \n وقال مكتب إحصاءات العمل التابع لوزارة العمل -في تقرير صدر اليوم الجمعة- إن القراءة الثابتة لمؤشر أسعار المنتجين للطلب النهائي الشهر الماضي جاءت بعد زيادة غير معدلة بلغت 0.2 في أغسطسآب الماضي. وعلى أساس سنوي، ارتفع المؤشر بنسبة 1.8، وهو أقل تقدم منذ فبرايرشباط الماضي. \n ويُظهر التقرير أن مؤشرا أقل تقلبا، يُستخدم لقياس التضخم باستثناء الغذاء والطاقة والتجارة، ارتفع بنسبة 0.1، مما يعادل أقل زيادة منذ مايوأيار 2023. في وقت ظهرت فيه البيانات الخاصة بالتضخم العام والقطاعات التي يعتمد عليها الاحتياطي الفدرالي لاتخاذ قراراته. \n وقد استقرت تكاليف الرعاية الطبية وتكاليف الرعاية الخارجية بالمستشفيات، في حين ارتفعت أسعار تذاكر الطيران بشكل حاد. \n توقع المتداولون أن يخفض الاحتياطي الفدرالي أسعار الفائدة ربع نقطة مئوية الشهر المقبل، بعد أن بدأ